In [33]:
!pip install sentencepiece datasets
!pip install wandb


In [38]:
import torch
from torch import nn
import numpy as np
import tensorflow as tf
from typing import Optional
import sentencepiece as spm
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from random import shuffle
import math
import wandb
from tqdm import tqdm
import torch.nn.functional as F

In [31]:
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: guhan-karthik25 (guhan-karthik25-kumaraguru-college-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [6]:
def generate_causal_mask(seq_len: int, device: Optional[torch.device] = None):
    """
    Additive mask (T, T) with 0 for allowed, -1e9 for blocked future positions.
    Suitable to add to attention logits.
    """
    mask = torch.triu(torch.ones(seq_len, seq_len, dtype=torch.bool), diagonal=1)
    mask = mask.float().masked_fill(mask, float("-1e9"))
    return mask.to(device) if device is not None else mask

In [7]:
class MultiHeadAttention(nn.Module):
    def __init__(self, input_dim, d_model, num_heads, max_len=2048):
        super().__init__()
        assert d_model % num_heads == 0
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads

        self.qkv = nn.Linear(input_dim, 3 * d_model)
        self.out_proj = nn.Linear(d_model, d_model)

        # For cross-attn
        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)

        # RoPE module
        self.rope = RotaryEmbedding(self.head_dim, max_len=max_len)

    def _apply_rope(self, q, k):
        # q, k: (B, H, T, Dh)
        return self.rope(q), self.rope(k)

    def _attention(self, q, k, v, mask=None):
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim)
        if mask is not None:
            if mask.dim() == 2:
                mask = mask.unsqueeze(0).unsqueeze(0)
            scores = scores + mask.to(scores.device)

        weights = torch.softmax(scores, dim=-1)
        return torch.matmul(weights, v)

    def forward(self, x, mask=None):
        B, T, _ = x.shape

        qkv = self.qkv(x)  # (B, T, 3D)
        qkv = qkv.view(B, T, self.num_heads, 3*self.head_dim).permute(0,2,1,3)
        q, k, v = qkv.chunk(3, dim=-1)

        # Apply RoPE
        q, k = self._apply_rope(q, k)

        out = self._attention(q, k, v, mask)
        out = out.permute(0,2,1,3).contiguous().view(B, T, -1)
        return self.out_proj(out)

    def forward_cross_attention(self, q_in, k_in, v_in, mask=None):
        B, Tq, _ = q_in.shape
        _, Tk, _ = k_in.shape

        q = self.q_linear(q_in).view(B, Tq, self.num_heads, self.head_dim).permute(0,2,1,3)
        k = self.k_linear(k_in).view(B, Tk, self.num_heads, self.head_dim).permute(0,2,1,3)
        v = self.v_linear(v_in).view(B, Tk, self.num_heads, self.head_dim).permute(0,2,1,3)

        # Apply RoPE to cross-attn q,k
        q, k = self._apply_rope(q, k)

        out = self._attention(q, k, v, mask)
        out = out.permute(0,2,1,3).contiguous().view(B, Tq, -1)
        return self.out_proj(out)


Positional Encoding
Rotary q,k embeddings



In [8]:
class RotaryEmbedding(nn.Module):
    def __init__(self, head_dim, max_len=2048):
        super().__init__()
        self.head_dim = head_dim

        # Compute base frequencies
        inv_freq = 1.0 / (10000 ** (torch.arange(0, head_dim, 2).float() / head_dim))

        # Precompute sin/cos tables
        t = torch.arange(max_len)
        freqs = torch.einsum("i,j->ij", t, inv_freq)

        emb = torch.cat((freqs, freqs), dim=-1)  # interleave
        self.register_buffer("cos_emb", emb.cos())
        self.register_buffer("sin_emb", emb.sin())

    def forward(self, x):
        """
        x: (B, H, T, D)
        apply RoPE on last dimension D
        """
        B, H, T, D = x.shape
        cos = self.cos_emb[:T].unsqueeze(0).unsqueeze(0)
        sin = self.sin_emb[:T].unsqueeze(0).unsqueeze(0)

        x1 = x[..., : D//2]
        x2 = x[..., D//2 :]

        # rotate
        x_rot = torch.cat([-x2, x1], dim=-1)
        return (x * cos) + (x_rot * sin)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=2048):
        super().__init__()

    def forward(self, x):
        return x   # RoPE replaces absolute positional embeddings


**Feed Forward Neural Network**
```
transform the data to higher dimension to capture complex information.
relu activation to introduce non-linearity.
transform back to lower dimension of d_model.
```

In [9]:
class FeedForwardNetwork(nn.Module):
    def __init__(self, d_model: int, d_ffn: int, activation=nn.ReLU()):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_model, d_ffn),
            activation,
            nn.Linear(d_ffn, d_model)
        )

    def forward(self, x):
        return self.net(x)



**Encoding layer**
```
structuring the
1.positional encoding
2.multihead attention
3.add and normalization
4.feed forward neural network

```

In [10]:
class EncodingLayer(nn.Module):
    def __init__(self, d_model: int, num_heads: int, d_ffn: int):
        super().__init__()
        self.ln1 = nn.LayerNorm(d_model)
        self.mha = MultiHeadAttention(d_model, d_model, num_heads)
        self.ln2 = nn.LayerNorm(d_model)
        self.ff = FeedForwardNetwork(d_model, d_ffn)

    def forward(self, x, src_mask: Optional[torch.Tensor] = None):
        # Pre-norm: LN -> MHA -> resid
        x = x + self.mha(self.ln1(x), mask=src_mask)
        # FFN block
        x = x + self.ff(self.ln2(x))
        return x



**Transformer Encoder**


one or more encoder layers

In [11]:
class Encoder(nn.Module):
    def __init__(self, vocab_size: int, d_model: int, num_layers: int, num_heads: int, d_ffn: int, max_len: int = 2048):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, d_model)
        self.pos_enc = PositionalEncoding(d_model, max_len=max_len)
        self.layers = nn.ModuleList([EncodingLayer(d_model, num_heads, d_ffn) for _ in range(num_layers)])
        self.ln = nn.LayerNorm(d_model)

    def forward(self, input_ids: torch.LongTensor, src_mask: Optional[torch.Tensor] = None):
        """
        input_ids: (B, T)
        returns (B, T, d_model)
        """
        x = self.token_emb(input_ids)  # (B, T, d_model)
        x = self.pos_enc(x)
        for layer in self.layers:
            x = layer(x, src_mask)
        return self.ln(x)

**Decoder layer**
```
structuring:
1. Positional encoder for decoder input.
2. multihead attention with mask
3. layer normalization.
4. multihead attention without mask with encoder output and decoder data as input
5. layer normalization
6. feed forward network
7. layer normalization.


In [12]:
class DecodingLayer(nn.Module):
    def __init__(self, d_model: int, num_heads: int, d_ffn: int):
        super().__init__()
        self.ln1 = nn.LayerNorm(d_model)
        self.self_att = MultiHeadAttention(d_model, d_model, num_heads)

        self.ln2 = nn.LayerNorm(d_model)
        self.cross_att = MultiHeadAttention(d_model, d_model, num_heads)

        self.ln3 = nn.LayerNorm(d_model)
        self.ff = FeedForwardNetwork(d_model, d_ffn)

    def forward(self, x: torch.Tensor, enc_output: Optional[torch.Tensor] = None, src_mask: Optional[torch.Tensor] = None, tgt_mask: Optional[torch.Tensor] = None):
        # masked self-attention (pre-norm)
        x = x + self.self_att(self.ln1(x), mask=tgt_mask)

        # cross-attention only if encoder output is provided
        if enc_output is not None:
            x = x + self.cross_att.forward_cross_attention(self.ln2(x), enc_output, enc_output, mask=src_mask)

        # feed-forward
        x = x + self.ff(self.ln3(x))
        return x



**Decoder**

In [13]:
class Decoder(nn.Module):
    def __init__(self, vocab_size: int, d_model: int, num_layers: int, num_heads: int, d_ffn: int, max_len: int = 2048):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, d_model)
        self.pos_enc = PositionalEncoding(d_model, max_len=max_len)
        self.layers = nn.ModuleList([DecodingLayer(d_model, num_heads, d_ffn) for _ in range(num_layers)])
        self.ln = nn.LayerNorm(d_model)

    def forward(self, input_ids: torch.LongTensor, enc_output: Optional[torch.Tensor] = None, src_mask: Optional[torch.Tensor] = None, tgt_mask: Optional[torch.Tensor] = None):
        B, T = input_ids.shape
        x = self.token_emb(input_ids)
        x = self.pos_enc(x)
        for layer in self.layers:
            x = layer(x, enc_output=enc_output, src_mask=src_mask, tgt_mask=tgt_mask)
        return self.ln(x)


Encoder only wrapper


In [14]:
class EncoderOnlyTransformer(nn.Module):
    def __init__(self, vocab_size: int, d_model: int, num_layers: int, num_heads: int, d_ffn: int, max_len: int = 2048):
        super().__init__()
        self.encoder = Encoder(vocab_size, d_model, num_layers, num_heads, d_ffn, max_len=max_len)
        self.ln = nn.LayerNorm(d_model)
        self.head = nn.Linear(d_model, vocab_size)

    def forward(self, input_ids: torch.LongTensor, src_mask: Optional[torch.Tensor] = None):
        enc = self.encoder(input_ids, src_mask=src_mask)
        enc = self.ln(enc)
        logits = self.head(enc)  # (B, T, V)
        return logits


Decoder only wrapper

In [15]:
class DecoderOnlyTransformer(nn.Module):
    def __init__(self, vocab_size: int, d_model: int, num_layers: int, num_heads: int, d_ffn: int, max_len: int = 2048):
        super().__init__()
        # build a decoder; token embedding sized to vocab
        self.decoder = Decoder(vocab_size, d_model, num_layers, num_heads, d_ffn, max_len=max_len)
        self.ln = nn.LayerNorm(d_model)
        self.head = nn.Linear(d_model, vocab_size)

    def forward(self, input_ids: torch.LongTensor):
        """
        input_ids: (B, T)
        returns logits: (B, T, V)
        """
        B, T = input_ids.shape
        tgt_mask = generate_causal_mask(T, device=input_ids.device)
        dec = self.decoder(input_ids, enc_output=None, src_mask=None, tgt_mask=tgt_mask)
        dec = self.ln(dec)
        logits = self.head(dec)
        return logits


In [16]:
def init_weights_xavier(module):
    if isinstance(module, nn.Linear):
        nn.init.xavier_uniform_(module.weight)
        if module.bias is not None:
            nn.init.zeros_(module.bias)
    elif isinstance(module, nn.Embedding):
        nn.init.normal_(module.weight, mean=0.0, std=0.02)
    elif isinstance(module, nn.LayerNorm):
        if getattr(module, "weight", None) is not None:
            nn.init.ones_(module.weight)
        if getattr(module, "bias", None) is not None:
            nn.init.zeros_(module.bias)

In [17]:
vocab_size = 2000
d_model = 128
num_layers = 4
num_heads = 4
d_ffn = 512
max_len = 256

In [72]:
model = DecoderOnlyTransformer(vocab_size, d_model, num_layers, num_heads, d_ffn, max_len=max_len)
model.apply(init_weights_xavier)

DecoderOnlyTransformer(
  (decoder): Decoder(
    (token_emb): Embedding(2000, 128)
    (pos_enc): PositionalEncoding()
    (layers): ModuleList(
      (0-3): 4 x DecodingLayer(
        (ln1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (self_att): MultiHeadAttention(
          (qkv): Linear(in_features=128, out_features=384, bias=True)
          (out_proj): Linear(in_features=128, out_features=128, bias=True)
          (q_linear): Linear(in_features=128, out_features=128, bias=True)
          (k_linear): Linear(in_features=128, out_features=128, bias=True)
          (v_linear): Linear(in_features=128, out_features=128, bias=True)
          (rope): RotaryEmbedding()
        )
        (ln2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (cross_att): MultiHeadAttention(
          (qkv): Linear(in_features=128, out_features=384, bias=True)
          (out_proj): Linear(in_features=128, out_features=128, bias=True)
          (q_linear): Linear(in_features=1

In [73]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

DecoderOnlyTransformer(
  (decoder): Decoder(
    (token_emb): Embedding(2000, 128)
    (pos_enc): PositionalEncoding()
    (layers): ModuleList(
      (0-3): 4 x DecodingLayer(
        (ln1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (self_att): MultiHeadAttention(
          (qkv): Linear(in_features=128, out_features=384, bias=True)
          (out_proj): Linear(in_features=128, out_features=128, bias=True)
          (q_linear): Linear(in_features=128, out_features=128, bias=True)
          (k_linear): Linear(in_features=128, out_features=128, bias=True)
          (v_linear): Linear(in_features=128, out_features=128, bias=True)
          (rope): RotaryEmbedding()
        )
        (ln2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (cross_att): MultiHeadAttention(
          (qkv): Linear(in_features=128, out_features=384, bias=True)
          (out_proj): Linear(in_features=128, out_features=128, bias=True)
          (q_linear): Linear(in_features=1

In [20]:
dataset = load_dataset("roneneldan/TinyStories")
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2119719
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 21990
    })
})

In [ ]:
corpus_path = "tinystories_corpus.txt"

with open(corpus_path, "w", encoding="utf-8") as f:
    for item in dataset["train"]:
        story = item["text"].replace("\n", " ")
        f.write(story.strip() + "\n")


In [ ]:
vocab_size = 2000
model_prefix = "tinystories_sp"

spm.SentencePieceTrainer.Train(
    input=corpus_path,
    model_prefix=model_prefix,
    vocab_size=vocab_size,
    character_coverage=1.0,
    model_type="bpe",
    pad_id=0,
    unk_id=1,
    bos_id=2,
    eos_id=3
)

In [23]:
sp = spm.SentencePieceProcessor()
sp.load(f"{model_prefix}.model")

True

In [40]:
class TinyStoriesDataset(Dataset):
    def __init__(self, hf_dataset, tokenizer, block_size=128):
        self.data = hf_dataset
        self.sp = tokenizer
        self.block_size = block_size

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data[idx]["text"]
        ids = self.sp.encode(text, out_type=int)

        # Always ensure at least block_size+1 tokens
        if len(ids) < self.block_size + 1:
            # pad the sequence
            needed = (self.block_size + 1) - len(ids)
            ids = ids + [0] * needed

        # SAFE: compute max starting index
        max_start = len(ids) - (self.block_size + 1)

        # If max_start == 0 → only one valid window
        if max_start == 0:
            start = 0
        else:
            start = torch.randint(0, max_start, (1,)).item()

        chunk = ids[start : start + self.block_size + 1]

        x = torch.tensor(chunk[:-1], dtype=torch.long)
        y = torch.tensor(chunk[1:], dtype=torch.long)
        return x, y


In [25]:
sp.encode("Hello! I am training a tiny transformer.", out_type=int)

[62,
 1692,
 1891,
 81,
 885,
 1438,
 37,
 6,
 1723,
 454,
 47,
 1867,
 1878,
 49,
 1064,
 1873]

In [41]:
block_size = 128
batch_size = 32

train_ds = TinyStoriesDataset(dataset["train"], sp, block_size=128)
val_ds = TinyStoriesDataset(dataset["validation"], sp, block_size=128)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)



In [28]:
inp, tgt = next(iter(train_loader))
inp = inp.to(device)
tgt = tgt.to(device)

logits = model(inp)   # (B, T, vocab_size)
loss = nn.CrossEntropyLoss()(logits.view(-1, vocab_size), tgt.view(-1))
loss


tensor(7.6714, device='cuda:0', grad_fn=<NllLossBackward0>)

In [75]:
config = {
    "epochs": 1,
    "batch_size": 32,
    "block_size": 128,
    "learning_rate": 3e-4,
    "weight_decay": 0.1,
    "warmup_steps": 200,
    "max_steps": 10000,  # safety stop
    "grad_clip": 1.0,
    "log_every": 100,
    "val_every": 1000,
}

wandb.init(project="tiny-gpt-training", config=config)

lr,▁▂▄▅▅██████████████████████████████████▇
step,▁▂▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇█████
train/loss,██▇▇▇▆▆▆▆▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁
val/loss,▁
lr,0.00028
step,2000
train/loss,3.1292
val/loss,3.48248


In [74]:
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=config["learning_rate"],
    weight_decay=config["weight_decay"]
)

# Cosine scheduler with warmup
def get_lr(step):
    if step < config["warmup_steps"]:
        return step / config["warmup_steps"]
    progress = (step - config["warmup_steps"]) / max(1, config["max_steps"] - config["warmup_steps"])
    return 0.5 * (1.0 + math.cos(math.pi * progress))



In [76]:
# ------------------------------
# Device + AMP scaler
# ------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
scaler = torch.cuda.amp.GradScaler()

# ------------------------------
# Tracking
# ------------------------------
best_val_loss = float("inf")
steps_since_improve = 0
patience = 1000            # stop if no improvement this long
global_step = 0

# ------------------------------
# TRAINING LOOP
# ------------------------------
for epoch in range(config["epochs"]):
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config['epochs']}")

    for inp, tgt in pbar:
        model.train()
        inp, tgt = inp.to(device), tgt.to(device)

        # ------------------------------
        # Learning rate scheduling
        # ------------------------------
        lr = config["learning_rate"] * get_lr(global_step)
        for param_group in optimizer.param_groups:
            param_group["lr"] = lr

        optimizer.zero_grad(set_to_none=True)

        # ------------------------------
        # Forward pass (mixed precision)
        # ------------------------------
        with torch.cuda.amp.autocast(dtype=torch.float16):
            logits = model(inp)
            loss = F.cross_entropy(
                logits.view(-1, logits.size(-1)),
                tgt.view(-1)
            )

        # Backward
        scaler.scale(loss).backward()

        # Gradient clipping
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), config["grad_clip"])

        scaler.step(optimizer)
        scaler.update()

        # ------------------------------
        # Logging to W&B
        # ------------------------------
        wandb.log({
            "train/loss": loss.item(),
            "lr": lr,
            "step": global_step
        })

        # Update progress bar
        pbar.set_postfix({"loss": f"{loss.item():.4f}"})

        # ------------------------------
        # PRINT EVERY log_every STEPS
        # ------------------------------
        if global_step % config["log_every"] == 0:
            print(f"\n[TRAIN] Step {global_step} | Loss: {loss.item():.4f} | LR: {lr:.6f}")

        # ------------------------------
        # VALIDATION
        # ------------------------------
        if global_step % config["val_every"] == 0 and global_step > 0:

            print(f"\n[VAL] Running validation at step {global_step}...")
            model.eval()
            val_losses = []

            with torch.no_grad():
                for v_inp, v_tgt in val_loader:
                    v_inp, v_tgt = v_inp.to(device), v_tgt.to(device)
                    with torch.cuda.amp.autocast(dtype=torch.float16):
                        v_logits = model(v_inp)
                        v_loss = F.cross_entropy(
                            v_logits.view(-1, v_logits.size(-1)),
                            v_tgt.view(-1)
                        )
                    val_losses.append(v_loss.item())

            mean_val_loss = sum(val_losses) / len(val_losses)

            print(f"[VAL] Step {global_step} | Val Loss: {mean_val_loss:.4f}")

            wandb.log({"val/loss": mean_val_loss, "step": global_step})

            # ------------------------------
            # SMART CHECKPOINTING
            # ------------------------------
            if mean_val_loss < best_val_loss:
                best_val_loss = mean_val_loss
                steps_since_improve = 0

                ckpt_path = f"best_model_step_{global_step}.pt"
                torch.save(model.state_dict(), ckpt_path)
                wandb.save(ckpt_path)

                print(f"✔ New Best Model Saved! Val loss improved to {best_val_loss:.4f}")
            else:
                steps_since_improve += config["val_every"]
                print(f"✘ No improvement for {steps_since_improve} steps")

            # Rolling checkpoint
            torch.save(model.state_dict(), "latest_model.pt")

            # ------------------------------
            # EARLY STOPPING
            # ------------------------------
            if steps_since_improve >= patience:
                print("\n############################")
                print("### EARLY STOPPING FIRED ###")
                print("### No improvement observed ###")
                print("############################\n")

                torch.save(model.state_dict(), "final_model.pt")
                wandb.finish()
                raise SystemExit("EARLY STOPPING TRIGGERED.")

        # ------------------------------
        # UPDATE STEP + STOP IF MAX REACHED
        # ------------------------------
        global_step += 1
        if global_step >= config["max_steps"]:
            break

    if global_step >= config["max_steps"]:
        break

# ------------------------------
# FINAL SAVE
# ------------------------------
torch.save(model.state_dict(), "final_model.pt")
print("\n🎉 Training Complete!")
wandb.finish()


/tmp/ipython-input-3915845230.py:5: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Epoch 1/1:   0%|          | 0/66242 [00:00<?, ?it/s]/tmp/ipython-input-3915845230.py:37: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.float16):
Epoch 1/1:   0%|          | 3/66242 [00:00<2:13:33,  8.27it/s, loss=7.6095]


[TRAIN] Step 0 | Loss: 7.6084 | LR: 0.000000


Epoch 1/1:   0%|          | 104/66242 [00:06<56:36, 19.47it/s, loss=6.4328]


[TRAIN] Step 100 | Loss: 6.4987 | LR: 0.000150


Epoch 1/1:   0%|          | 203/66242 [00:11<49:54, 22.05it/s, loss=5.4134]

...

Epoch 1/1:  15%|█▍        | 9904/66242 [11:41<43:39, 21.50it/s, loss=2.5456]


[TRAIN] Step 9900 | Loss: 2.4251 | LR: 0.000000


Epoch 1/1:  15%|█▌        | 9999/66242 [11:46<1:06:13, 14.15it/s, loss=2.5031]



🎉 Training Complete!


lr,███████▇▇▇▆▆▆▆▆▆▆▆▆▆▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁
step,▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇██
train/loss,██▆▆▅▅▄▄▄▃▃▂▂▃▂▂▃▃▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▁▁▂▁
val/loss,█▅▃▃▂▂▁▁▁
lr,0.0
step,9999
train/loss,2.5031
val/loss,2.49107
